In [1]:
import pandas as pd
import json
import numpy as np
import os

from tqdm import tqdm
from typing import List, Dict

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.core.lightning import LightningModule

from sklearn.metrics import ndcg_score, roc_auc_score

from argparse import ArgumentParser, Namespace

from data import load_data


%load_ext autoreload
%autoreload 2

/home/nikita/miniconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


## Data Loading

In [2]:
args = Namespace(**{'data_root': './data/', 'cell_type': 'E016',
                    'n_bins': 100, 'batch_size': 16, 'n_hms': 5, 
                    'bin_rnn_size': 32, 'bidirectional': True,
                    'num_layers': 1, 'dropout': 0.5, 'lr': 0.0002, 'clip': 1})

In [3]:
Train, Valid, Test = load_data(args)

==>loading train data
Number of genes: 6601
Number of entries: 660100
Number of HMs: 7
==>loading valid data
Number of genes: 6601
Number of entries: 660100
Number of HMs: 7
==>loading test data
Number of genes: 6600
Number of entries: 660000
Number of HMs: 7


In [4]:
columns = ['GeneID', 'Bin ID', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3', 'H3K9me3', 'Label']
train = pd.read_csv('./data/E011/classification/train.csv', header=None)
train.columns = columns

test = pd.read_csv('./data/E011/classification/test.csv', header=None)
test.columns = columns

train.head()

,GeneID,Bin ID,H3K27me3,H3K36me3,H3K4me1,H3K4me3,H3K9me3,Label
0,3,1,1,4,3,12,1,1
1,3,2,1,2,5,8,0,1
2,3,3,1,2,8,14,1,1
3,3,4,1,2,6,12,1,1
4,3,5,0,1,4,8,1,1


## CNN-based

In [11]:
class ChromePlusCNN(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        self.conv1 = nn.Conv1d(5, 64, kernel_size=5)
        self.maxpool = nn.MaxPool1d(10)
        self.linear1 = nn.Linear(576, 256)
        self.linear_out = nn.Linear(256, 1)

    def forward(self, X):
        X = self.conv1(X.transpose(2, 1))
        X = self.maxpool(X)
        X = X.view(-1, 64 * 9)
        return self.linear_out(nn.ReLU()(self.linear1(X)))

    def training_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}

    def validation_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'val_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'val_loss': avg_loss, 'val_roc_auc': roc_auc}
        return logs
    
    def test_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'test_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'test_loss': avg_loss, 'test_roc_auc': roc_auc}
        return logs
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [12]:
cnn = ChromePlusCNN(args)

MAX_EPOCHS = 10
trainer = Trainer(gpus=[0], max_epochs=MAX_EPOCHS)
trainer.fit(cnn, Train, Valid)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type      | Params
-----------------------------------------
0 | conv1      | Conv1d    | 1 K   
1 | maxpool    | MaxPool1d | 0     
2 | linear1    | Linear    | 147 K 
3 | linear_out | Linear    | 257   
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [13]:
trainer.test(cnn, Test)

/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.4437, device='cuda:0'),
 'test_roc_auc': 0.7689540209550758}
--------------------------------------------------------------------------------



{'test_loss': 0.44372010231018066, 'test_roc_auc': 0.7689540209550758}

## LSTM-based

In [14]:
class ChromePlusLSTM(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        self.lstm = nn.LSTM(5, 128, bidirectional=True)
        self.linear_out = nn.Linear(256, 1)

    def forward(self, X):
        h, _ = self.lstm(X)
        h, _ = torch.max(h, 1)
        return self.linear_out(h)

    def training_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}

    def validation_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'val_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'val_loss': avg_loss, 'val_roc_auc': roc_auc}
        return logs
    
    def test_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'test_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'test_loss': avg_loss, 'test_roc_auc': roc_auc}
        return logs
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [15]:
lstm = ChromePlusLSTM(args)

MAX_EPOCHS = 10
trainer = Trainer(gpus=[0], max_epochs=MAX_EPOCHS)
trainer.fit(lstm, Train, Valid)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type   | Params
--------------------------------------
0 | lstm       | LSTM   | 138 K 
1 | linear_out | Linear | 257   
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [16]:
trainer.test(lstm, Test)

/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.4285, device='cuda:0'),
 'test_roc_auc': 0.7889610293037641}
--------------------------------------------------------------------------------



{'test_loss': 0.428501695394516, 'test_roc_auc': 0.7889610293037641}

## Self-attention-based

In [140]:
class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):

        attn = torch.bmm(q, k.transpose(1, 2))
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(mask, -np.inf)

        attn = self.softmax(attn)
        attn = self.dropout(attn)
        output = torch.bmm(attn, v)

        return output, attn
    

class MultiHeadAttention(nn.Module):
    ''' Multi-Head Attention module '''

    def __init__(self, n_head, d_model, d_k, d_v, dropout=0.1):
        super().__init__()

        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k)
        self.w_ks = nn.Linear(d_model, n_head * d_k)
        self.w_vs = nn.Linear(d_model, n_head * d_v)
        nn.init.normal_(self.w_qs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_ks.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_k)))
        nn.init.normal_(self.w_vs.weight, mean=0, std=np.sqrt(2.0 / (d_model + d_v)))

        self.attention = ScaledDotProductAttention(temperature=np.power(d_k, 0.5))
        self.layer_norm = nn.LayerNorm(d_model)

        self.fc = nn.Linear(n_head * d_v, d_model)
        nn.init.xavier_normal_(self.fc.weight)

        self.dropout = nn.Dropout(dropout)


    def forward(self, q, k, v, mask=None):
        
        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        
        sz_b, len_q, _ = q.size()
        sz_b, len_k, _ = k.size()
        sz_b, len_v, _ = v.size()
        
        residual = q
        
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)
        
        q = q.permute(2, 0, 1, 3).contiguous().view(-1, len_q, d_k) # (n*b) x lq x dk
        k = k.permute(2, 0, 1, 3).contiguous().view(-1, len_k, d_k) # (n*b) x lk x dk
        v = v.permute(2, 0, 1, 3).contiguous().view(-1, len_v, d_v) # (n*b) x lv x dv
        
        output, attn = self.attention(q, k, v)
        
        output = output.view(n_head, sz_b, len_q, d_v)
        output = output.permute(1, 2, 0, 3).contiguous().view(sz_b, len_q, -1) # b x lq x (n*dv)
        
        output = self.dropout(self.fc(output))
        output = self.layer_norm(output + residual)
        
        return output, attn

In [141]:
class ChromePlusSelfAttention(LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        self.multi_head_attention_h = MultiHeadAttention(4, 5, 1024, 100)
        self.linear_out = nn.Linear(5, 1)

    def forward(self, X):
        h, _ = self.multi_head_attention_h(X, X, X)
        h, _ = torch.max(h, 1)
        h = self.linear_out(h)
        return h

    def training_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        logs = {'train_loss': loss}
        return {'loss': loss, 'log': logs}

    def validation_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'val_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'val_loss': avg_loss, 'val_roc_auc': roc_auc}
        return logs
    
    def test_step(self, batch, batch_idx):
        X = batch['input']
        y_true = batch['label'].unsqueeze(1).float()
        y_pred = self(X)
        loss = F.binary_cross_entropy_with_logits(y_pred, y_true)
        return {
            'test_loss': loss,
            'logits': y_pred.detach().cpu(),
            'labels': y_true.detach().cpu()
        }

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        y_true = torch.cat([x['labels'] for x in outputs])
        y_pred = torch.sigmoid(torch.cat([x['logits'] for x in outputs]))
        roc_auc = roc_auc_score(y_true, y_pred)
        logs = {'test_loss': avg_loss, 'test_roc_auc': roc_auc}
        return logs
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.lr)

In [142]:
attention = ChromePlusSelfAttention(args)

MAX_EPOCHS = 10
trainer = Trainer(gpus=[0], max_epochs=MAX_EPOCHS)
trainer.fit(attention, Train, Valid)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type               | Params
--------------------------------------------------------------
0 | multi_head_attention_h | MultiHeadAttention | 53 K  
1 | linear_out             | Linear             | 6     
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


1

In [143]:
trainer.test(attention, Test)

/home/nikita/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
TEST RESULTS
{'test_loss': tensor(0.4368, device='cuda:0'),
 'test_roc_auc': 0.7890322034188788}
--------------------------------------------------------------------------------



{'test_loss': 0.43679988384246826, 'test_roc_auc': 0.7890322034188788}